# Image Features
We have seen that we can achieve reasonable performance on an image classification task by training a linear classifier on the pixels of the input image. In this exercise we will show that we can improve our classification performance by training linear classifiers not on raw pixels but on features that are computed from the raw pixels.

In [1]:
import random
import numpy as np
from exercise_code.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Load data
Similar to previous exercises, we will load CIFAR-10 data from disk. However, we don't apply our previous preprocessing steps, as we want to extract custom features later on.

In [2]:
from exercise_code.data_utils import load_CIFAR10
from exercise_code.vis_utils import visualize_cifar10

def get_CIFAR10_data(num_training=48000, num_validation=1000, num_test=1000):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for feature extraction and training.
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = '../datasets/cifar10'
    X, y = load_CIFAR10(cifar10_dir)
    

    # Our training set will be the first num_train points from the original dataset.
    mask = range(num_training)
    X_train = X[mask]
    y_train = y[mask]
    
    # Our validation set will be num_validation points from the original dataset.
    mask = range(num_training, num_training + num_validation)
    X_val = X[mask]
    y_val = y[mask]
    
    # Our test set will be num_test points from the original dataset.
    mask = range(num_training + num_validation, num_training + num_validation + num_test)
    X_test = X[mask]
    y_test = y[mask]

    return X, y, X_train, y_train, X_val, y_val, X_test, y_test

# Invoke the above function to get our data.
X_raw, y_raw, X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

# visualize raw data
visualize_cifar10(X_raw, y_raw)

Train data shape:  (48000, 32, 32, 3)
Train labels shape:  (48000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (1000, 32, 32, 3)
Test labels shape:  (1000,)


## Extract Features
For each image we will compute a Histogram of Oriented
Gradients (HOG) as well as a color histogram using the hue channel in HSV
color space. We form our final feature vector for each image by concatenating
the HOG and color histogram feature vectors.

Roughly speaking, HOG should capture the texture of the image while ignoring
color information, and the color histogram represents the color of the input
image while ignoring texture. As a result, we expect that using both together
ought to work better than using either alone.

The `hog_feature` and `color_histogram_hsv` functions both operate on a single
image and return a feature vector for that image. The extract_features
function takes a set of images and a list of feature functions and evaluates
each feature function on each image, storing the results in a matrix where
each column is the concatenation of all feature vectors for a single image.

In [3]:
from exercise_code.features import *

num_color_bins = 10 # Number of bins in the color histogram
feature_fns = [hog_feature, lambda img: color_histogram_hsv(img, nbin=num_color_bins)]
X_train_feats = extract_features(X_train, feature_fns, verbose=True)
X_val_feats = extract_features(X_val, feature_fns)
X_test_feats = extract_features(X_test, feature_fns)

# Preprocessing: Subtract the mean feature
mean_feat = np.mean(X_train_feats, axis=0, keepdims=True)
X_train_feats -= mean_feat
X_val_feats -= mean_feat
X_test_feats -= mean_feat

# Preprocessing: divide by standard deviation. This ensures that each feature
# has roughly the same scale.
std_feat = np.std(X_train_feats, axis=0, keepdims=True)
X_train_feats /= std_feat
X_val_feats /= std_feat
X_test_feats /= std_feat

# Preprocessing: add a bias dimension
X_train_feats = np.hstack([X_train_feats, np.ones((X_train_feats.shape[0], 1))])
X_val_feats = np.hstack([X_val_feats, np.ones((X_val_feats.shape[0], 1))])
X_test_feats = np.hstack([X_test_feats, np.ones((X_test_feats.shape[0], 1))])

Done extracting features for 1000/48000 images
Done extracting features for 2000/48000 images
Done extracting features for 3000/48000 images
Done extracting features for 4000/48000 images
Done extracting features for 5000/48000 images
Done extracting features for 6000/48000 images
Done extracting features for 7000/48000 images
Done extracting features for 8000/48000 images
Done extracting features for 9000/48000 images
Done extracting features for 10000/48000 images
Done extracting features for 11000/48000 images
Done extracting features for 12000/48000 images
Done extracting features for 13000/48000 images
Done extracting features for 14000/48000 images
Done extracting features for 15000/48000 images
Done extracting features for 16000/48000 images
Done extracting features for 17000/48000 images
Done extracting features for 18000/48000 images
Done extracting features for 19000/48000 images
Done extracting features for 20000/48000 images
Done extracting features for 21000/48000 images
D

## Train a Softmax classifier on features
Using the multiclass Softmax code developed earlier in the assignment, train a Softmax classifier on top of the features extracted above; this should achieve better results than training the classifier directly on top of raw pixels.

In [4]:
# Use the validation set to tune the learning rate and regularization strength. You can 
# reuse the softmax_hyperparameter_tuning that you implemented in
# exercise_code.classifiers.softmax

from exercise_code.classifiers.softmax import SoftmaxClassifier, softmax_hyperparameter_tuning

best_softmax = None

################################################################################
# TODO: Train a softmax classifier on the features and tune hyperparameters to #
# obtain a strong model. You can alter and reuse the                           #
# softmax_hyperparameter_tuning function. Store your best network in the       #
# best_softmax variable.                                                       #
################################################################################
best_val = -1
results = {}
learning_rates = [1e-7, 5e-7, 1e-8]
regularization_strengths = [1e4, 1e5, 1e6]

# Get the combinations of the learning rates and regularizations strengths.
combinations = np.array(np.meshgrid(learning_rates,regularization_strengths)).T.reshape(-1,2)
for i,comb in enumerate(combinations):

    print("[ %",i*100/len(combinations)," ]Trying combination: ", comb)

    # Create and train the model
    softmax_classifier = SoftmaxClassifier()
    softmax_classifier.train(X_train_feats,y_train,learning_rate=comb[0],reg=comb[1],num_iters=1500)

    # Get training accuracy
    y_train_pred = softmax_classifier.predict(X_train_feats)
    [training_accuracy,] = (np.mean(y_train == y_train_pred), )

    # Get validation accuracy
    y_validation_pred = softmax_classifier.predict(X_val_feats)
    [validation_accuracy,] = (np.mean(y_val == y_validation_pred), )

    # Save the results
    results[(comb[0],comb[1])] = (training_accuracy,validation_accuracy)
        
    # Save best validation accuracy
    if best_val < validation_accuracy:
        best_val = validation_accuracy
        best_softmax = softmax_classifier

# Print out results.
for (lr, reg) in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
          lr, reg, train_accuracy, val_accuracy))

print('best validation accuracy achieved during validation: %f' % best_val)
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

[ % 0.0  ]Trying combination:  [1.e-07 1.e+04]
[ % 11.11111111111111  ]Trying combination:  [1.e-07 1.e+05]
[ % 22.22222222222222  ]Trying combination:  [1.e-07 1.e+06]
[ % 33.333333333333336  ]Trying combination:  [5.e-07 1.e+04]
[ % 44.44444444444444  ]Trying combination:  [5.e-07 1.e+05]
[ % 55.55555555555556  ]Trying combination:  [5.e-07 1.e+06]
[ % 66.66666666666667  ]Trying combination:  [1.e-08 1.e+04]
[ % 77.77777777777777  ]Trying combination:  [1.e-08 1.e+05]
[ % 88.88888888888889  ]Trying combination:  [1.e-08 1.e+06]
lr 1.000000e-08 reg 1.000000e+04 train accuracy: 0.091125 val accuracy: 0.092000
lr 1.000000e-08 reg 1.000000e+05 train accuracy: 0.091021 val accuracy: 0.084000
lr 1.000000e-08 reg 1.000000e+06 train accuracy: 0.412979 val accuracy: 0.414000
lr 1.000000e-07 reg 1.000000e+04 train accuracy: 0.110875 val accuracy: 0.114000
lr 1.000000e-07 reg 1.000000e+05 train accuracy: 0.416271 val accuracy: 0.423000
lr 1.000000e-07 reg 1.000000e+06 train accuracy: 0.399313 v

In [5]:
# Evaluate your trained classifier on the test set
y_test_pred = best_softmax.predict(X_test_feats)
test_accuracy = np.mean(y_test == y_test_pred)
print(test_accuracy)

0.397


In [6]:
# An important way to gain intuition about how an algorithm works is to
# visualize the mistakes that it makes. In this visualization, we show examples
# of images that are misclassified by our current system. The first column
# shows images that our system labeled as "plane" but whose true label is
# something other than "plane".

examples_per_class = 8
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for cls, cls_name in enumerate(classes):
    idxs = np.where((y_test != cls) & (y_test_pred == cls))[0]
    if len(idxs) > 0:
        idxs = np.random.choice(idxs, min(examples_per_class, len(idxs)), replace=False)
    else:
        idxs = []
    for i, idx in enumerate(idxs):
        plt.subplot(examples_per_class, len(classes), i * len(classes) + cls + 1)
        plt.imshow(X_test[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls_name)
plt.show()

<div class="alert alert-info">
    <h3>Inline Question</h3>
    <p>Describe the misclassification results that you see. Do they make sense?</p>
</div>

## Neural Network on image features
Earlier in this assigment we saw that training a two-layer neural network on raw pixels achieved better classification performance than linear classifiers on raw pixels. In this notebook we have seen that linear classifiers on image features outperform linear classifiers on raw pixels. 

For completeness, we should also try training a neural network on image features. This approach should outperform all previous approaches: you should easily be able to achieve over 55% classification accuracy on the test set.

Note that in the neural net class we have used explicit biases. In the preprocessing of this exercise we have also added a bias dimension to the input, such that we don't have to consider an explicit bias in the linear classifier above. The neural net therefore has an additional parameter in the first layer (the additional bias term), which after all does not matter, as both bias terms can be learned and behave almost the same as having a single bias term.

In [8]:
from exercise_code.classifiers.neural_net import TwoLayerNet, neuralnetwork_hyperparameter_tuning

input_dim = X_train_feats.shape[1]
hidden_dim = 500
num_classes = 10

net = TwoLayerNet(input_dim, hidden_dim, num_classes)
best_net = None

################################################################################
# TODO: Train a two-layer neural network on image features. You may want to    #
# validate various parameters as in previous sections. Store your best         #
# model in the best_net variable.                                              #
################################################################################

best_val = -1
best_comb = None


learning_rates = [40e-2, 50e-2,60e-2,70e-2,80e-2,90e-2,1]
regularization_strengths = [1e-3,1e-2,1e-1]
num_epochs = [2000,5000]
hidden_dims = [200,100,500]

combinations = np.array(np.meshgrid(learning_rates, regularization_strengths, num_epochs, hidden_dims)).T.reshape(-1,4)
for i,combination in enumerate(combinations):
    learning_rate = combination[0]
    regularization_strength = combination[1]
    num_epoch = combination[2]
    hidden_d = int(combination[3])
    print('[ %% %f ] - Combination learning rate: [ %f ] reg: [ %f ] hidden: [ %d ] epoch: [ %d ]' % 
    ((i*100/len(combinations)),learning_rate,regularization_strength,hidden_d, num_epoch))

    net = TwoLayerNet(input_dim,hidden_d,num_classes)
    res = net.train(X_train_feats,y_train, X_val_feats, y_val, learning_rate=learning_rate,reg=regularization_strength,num_iters=int(num_epoch))

    if res['val_acc_history'][-1] > best_val:
        best_val = res['val_acc_history'][-1]
        best_net = net
        best_comb = combination
    print('Training Accuracy: [ %f ] Validation Accuracy : [ %f ]' % (res['train_acc_history'][-1],res['val_acc_history'][-1]))

print('Best results are achieved with params: ', best_comb)

################################################################################
#                              END OF YOUR CODE                                #
################################################################################

[ % 0.000000 ] - Combination learning rate: [ 0.400000 ] reg: [ 0.001000 ] hidden: [ 200 ] epoch: [ 2000 ]
Training Accuracy: [ 0.740000 ] Validation Accuracy : [ 0.593000 ]
[ % 0.793651 ] - Combination learning rate: [ 0.400000 ] reg: [ 0.010000 ] hidden: [ 200 ] epoch: [ 2000 ]
Training Accuracy: [ 0.650000 ] Validation Accuracy : [ 0.547000 ]
[ % 1.587302 ] - Combination learning rate: [ 0.400000 ] reg: [ 0.100000 ] hidden: [ 200 ] epoch: [ 2000 ]
Training Accuracy: [ 0.475000 ] Validation Accuracy : [ 0.413000 ]
[ % 2.380952 ] - Combination learning rate: [ 0.500000 ] reg: [ 0.001000 ] hidden: [ 200 ] epoch: [ 2000 ]
Training Accuracy: [ 0.845000 ] Validation Accuracy : [ 0.572000 ]
[ % 3.174603 ] - Combination learning rate: [ 0.500000 ] reg: [ 0.010000 ] hidden: [ 200 ] epoch: [ 2000 ]
Training Accuracy: [ 0.660000 ] Validation Accuracy : [ 0.561000 ]
[ % 3.968254 ] - Combination learning rate: [ 0.500000 ] reg: [ 0.100000 ] hidden: [ 200 ] epoch: [ 2000 ]
Training Accuracy: [ 0.

Training Accuracy: [ 0.495000 ] Validation Accuracy : [ 0.353000 ]
[ % 38.095238 ] - Combination learning rate: [ 0.600000 ] reg: [ 0.001000 ] hidden: [ 100 ] epoch: [ 2000 ]
Training Accuracy: [ 0.805000 ] Validation Accuracy : [ 0.579000 ]
[ % 38.888889 ] - Combination learning rate: [ 0.600000 ] reg: [ 0.010000 ] hidden: [ 100 ] epoch: [ 2000 ]
Training Accuracy: [ 0.635000 ] Validation Accuracy : [ 0.540000 ]
[ % 39.682540 ] - Combination learning rate: [ 0.600000 ] reg: [ 0.100000 ] hidden: [ 100 ] epoch: [ 2000 ]
Training Accuracy: [ 0.480000 ] Validation Accuracy : [ 0.415000 ]
[ % 40.476190 ] - Combination learning rate: [ 0.700000 ] reg: [ 0.001000 ] hidden: [ 100 ] epoch: [ 2000 ]
Training Accuracy: [ 0.820000 ] Validation Accuracy : [ 0.581000 ]
[ % 41.269841 ] - Combination learning rate: [ 0.700000 ] reg: [ 0.010000 ] hidden: [ 100 ] epoch: [ 2000 ]
Training Accuracy: [ 0.615000 ] Validation Accuracy : [ 0.546000 ]
[ % 42.063492 ] - Combination learning rate: [ 0.700000 ] 

Training Accuracy: [ 0.610000 ] Validation Accuracy : [ 0.525000 ]
[ % 75.396825 ] - Combination learning rate: [ 0.700000 ] reg: [ 0.100000 ] hidden: [ 500 ] epoch: [ 2000 ]
Training Accuracy: [ 0.485000 ] Validation Accuracy : [ 0.402000 ]
[ % 76.190476 ] - Combination learning rate: [ 0.800000 ] reg: [ 0.001000 ] hidden: [ 500 ] epoch: [ 2000 ]
Training Accuracy: [ 0.885000 ] Validation Accuracy : [ 0.583000 ]
[ % 76.984127 ] - Combination learning rate: [ 0.800000 ] reg: [ 0.010000 ] hidden: [ 500 ] epoch: [ 2000 ]
Training Accuracy: [ 0.715000 ] Validation Accuracy : [ 0.549000 ]
[ % 77.777778 ] - Combination learning rate: [ 0.800000 ] reg: [ 0.100000 ] hidden: [ 500 ] epoch: [ 2000 ]
Training Accuracy: [ 0.530000 ] Validation Accuracy : [ 0.389000 ]
[ % 78.571429 ] - Combination learning rate: [ 0.900000 ] reg: [ 0.001000 ] hidden: [ 500 ] epoch: [ 2000 ]
Training Accuracy: [ 0.910000 ] Validation Accuracy : [ 0.605000 ]
[ % 79.365079 ] - Combination learning rate: [ 0.900000 ] 

In [ ]:
# Run your neural net classifier on the test set. You should be able to
# get more than 55% accuracy.
test_acc = (best_net.predict(X_test_feats) == y_test).mean()
print("Test accuracy: ", test_acc)

## Save the model

When you are satisfied with your training you can save your model. 
You need to get a score of at least **55%** to pass this submission.
<div class="alert alert-danger">
    <h3>Warning</h3>
    <p>You might get an error like this:</p>
    <p>PicklingError: Can't pickle ...</p>
    <p>The reason is that we are using autoreload and working on this class during the notebook session. If you get this error simply restart the kernel and rerun the whole script (Kernel -> Restart & Run All) or only the important cells for generating your model.</p>
</div>

In [ ]:
from exercise_code.model_savers import save_feature_neural_net
save_feature_neural_net(best_net)